# Inference and Reasoning with Bayesian Networks

###### COMP4670/8600 - Introduction to Statistical Machine Learning - Assignment 2

Name: Sina Eghbal

Student ID: u5544352

## Instructions

|             |Notes|
|:------------|:--|
|Maximum marks| 19|
|Weight|19% of final grade|
|Format| Complete this ipython notebook. Do not forget to fill in your name and student ID above|
|Submission mode| Use [wattle](https://wattle.anu.edu.au/)|
|Formulas| All formulas which you derive need to be explained unless you use very common mathematical facts. Picture yourself as explaining your arguments to somebody who is just learning about your assignment. With other words, do not assume that the person marking your assignment knows all the background and therefore you can just write down the formulas without any explanation. It is your task to convince the reader that you know what you are doing when you derive an argument. Typeset all formulas in $\LaTeX$.|
| Code quality | Python code should be well structured, use meaningful identifiers for variables and subroutines, and provide sufficient comments. Please refer to the examples given in the tutorials. |
| Code efficiency | An efficient implementation of an algorithm uses fast subroutines provided by the language or additional libraries. For the purpose of implementing Machine Learning algorithms in this course, that means using the appropriate data structures provided by Python and in numpy/scipy (e.g. Linear Algebra and random generators). |
| Cooperation | All assignments must be done individually. Cheating and plagiarism will be dealt with in accordance with University procedures (please see the ANU policies on [Academic Honesty and Plagiarism](http://academichonesty.anu.edu.au)). Hence, for example, code for programming assignments must not be developed in groups, nor should code be shared. You are encouraged to broadly discuss ideas, approaches and techniques with a few other students, but not at a level of detail where specific solutions or implementation issues are described by anyone. If you choose to consult with other students, you will include the names of your discussion partners for each solution. If you have any questions on this, please ask the lecturer before you act. |

$\newcommand{\dotprod}[2]{\left\langle #1, #2 \right\rangle}$
$\newcommand{\onevec}{\mathbb{1}}$

Setting up the environment (there is some hidden latex which needs to be run in this cell).

In [1]:
import itertools, copy
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, Image
from collections import OrderedDict

%matplotlib inline

## Part 1: Graphical Models

### Problem setting

We are interested to predict the outcome of the election in an imaginary country, called Under Some Assumptions (USA). There are four candidates for whom the citizens can **Vote** for: Bernie, Donald, Hillary, and Ted. The citizens live in four **Region**s: north, south, east and west. We have general demographic information about the people, namely: **Gender** (male, female) and **Hand**edness (right, left). Based on surveys done by an external company, we believe that the **Region** and **Gender** affects whether the people use their **Jacket**s full time, part time or never. Surprisingly, the company told us that the **Age** of their shoes (new, worn, old) depends on how often they wear their **Jacket**s. Furthermore, the **Gender** and their preferred **Hand** affects the **Colour** of their hat (white, black). Finally, surveys say that the citizens will **Vote** based on their **Region**, **Age** of their shoes and **Colour** of their hats.

The directed graphical model is depicted below:

In [2]:
Image(url="https://machlearn.gitlab.io/isml2017/assignments/election_model.png")

### Conditional probability tables

After paying the survey firm some more money, they provided the following conditional probability tables.

|$p(R)$ | R=n | R=s | R=e | R=w |
|:-----:|:--:|:--:|:--:|:--:|
|marginal| 0.2 | 0.1 | 0.5 | 0.2 |

|$p(G)$ | G=m | G=f |
|:-----:|:--:|:--:|
|marginal| 0.3 | 0.7 |

|$p(H)$ | H=r | H=l |
|:-----:|:--:|:--:|
|marginal| 0.9 | 0.1 |

| $p(J|R,G)$ | R=n,G=m | R=n,G=f | R=s,G=m | R=s,G=f | R=e,G=m | R=e,G=f | R=w,G=m | R=w,G=f |
|:-----:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
|**J**=full $\quad$  |0.9 |0.8 |0.1 | 0.3 |0.4 |0.01| 0.02 | 0.2  |
|**J**=part $\quad$  |0.08|0.17|0.03| 0.35|0.05|0.01| 0.2  | 0.08 |
|**J**=never $\quad$ |0.02|0.03|0.87| 0.35|0.55|0.98| 0.78 | 0.72 |

| $p(A|J)$ | J=full | J=part | J=never |
|:-----:|:--:|:--:|:--:|
|**A**=new  |0.01|0.96|0.3|
|**A**=worn |0.98|0.03|0.5|
|**A**=old  |0.01|0.01|0.2|

| $p(C|G,H)$ | G=m,H=r | G=m,H=l | G=f,H=r | G=f,H=l |
|:-----:|:--:|:--:|:--:|:--:|
|**C**=black $\quad$ |0.9 |0.83 |0.17 | 0.3 |
|**C**=white $\quad$ |0.1 |0.17|0.83 | 0.7 |

The final conditional probability table is given by the matrix below. The order of the rows and columns are also given below.

In [3]:
vote_column_names = ['north,new,black', 'north,new,white', 'north,worn,black', 'north,worn,white', 
                'north,old,black', 'north,old,white', 'south,new,black', 'south,new,white', 
                'south,worn,black', 'south,worn,white', 'south,old,black', 'south,old,white', 
                'east,new,black', 'east,new,white', 'east,worn,black', 'east,worn,white', 
                'east,old,black', 'east,old,white', 'west,new,black', 'west,new,white', 
                'west,worn,black', 'west,worn,white', 'west,old,black', 'west,old,white']

vote_outcomes = ('bernie','donald','hillary','ted')

vote_pmf_array = np.array(
        [
            [0.1,0.1,0.4,0.02,0.2,0.1,0.1,0.04,0.2,0.1,0.1 ,0.1,0.4 ,0.1 ,0.1,0.1 ,0.1,0.04,0.3,0.2,0.1,0.3,0.34,0.35],
            [0.3,0.4,0.2,0.5 ,0.1,0.2,0.1,0.5 ,0.1,0.2,0.5 ,0.3,0.2 ,0.42,0.2,0.67,0.4,0.4 ,0.1,0.1,0.5,0.1,0.1 ,0.1],
            [0.5,0.4,0.3,0.3 ,0.5,0.6,0.6,0.3 ,0.5,0.4,0.36,0.3,0.28,0.3 ,0.4,0.1 ,0.4,0.16,0.4,0.2,0.3,0.3,0.4 ,0.5],
            [0.1,0.1,0.1,0.18,0.2,0.1,0.2,0.16,0.2,0.3,0.04,0.3,0.12,0.18,0.3,0.13,0.1,0.4 ,0.2,0.5,0.1,0.3,0.16,0.05],
        ]
)

The 7 conditional probability tables in are encoded in python below. 

**Base your subsequent computations on these objects**.

In [4]:
class RandomVariable(object):
    def __init__(self, name, parents, outcomes, pmf_array):
        assert isinstance(name, str)
        assert all(isinstance(_, RandomVariable) for _ in parents)
        assert isinstance(outcomes, tuple)
        assert all(isinstance(_, str) for _ in outcomes)
        assert isinstance(parents, tuple)
        assert isinstance(pmf_array, np.ndarray)
        keys = tuple(map(tuple, itertools.product(*[_.outcomes for _ in parents])))
        assert np.allclose(np.sum(pmf_array, 0), 1)
        expected_shape = (len(outcomes), len(keys))
        assert pmf_array.shape == expected_shape, (pmf_array.shape, expected_shape)
        pmfs = {k: {outcome: probability for outcome, probability in zip(outcomes, probabilities)} 
                for k, probabilities in zip(keys, pmf_array.T)}
        self.name, self.parents, self.outcomes, self.pmfs = name, parents, outcomes, pmfs
    
    def __str__ (self):
        return self.name + " -> " + str(self.outcomes)
    
    def __repr__ (self):
        return self.name + " -> " + str(self.outcomes)


class BayesianNetwork(object):
    def __init__(self, *random_variables):
        assert all(isinstance(_, RandomVariable) for _ in random_variables)
        self.random_variables = random_variables

region_pmf_array = np.array([[0.2, 0.1, 0.5, 0.2]]).T
region = RandomVariable(
    name='region',
    parents=tuple(),
    outcomes=('north', 'south', 'east', 'west'), 
    pmf_array = region_pmf_array,
)

gender_pmf_array = np.array([[0.3, 0.7]]).T
gender = RandomVariable(
    name='gender',
    parents=tuple(),
    outcomes=('male', 'female'), 
    pmf_array = gender_pmf_array
)

hand_pmf_array = np.array([[0.9, 0.1]]).T
hand = RandomVariable(
    name='hand',
    parents=tuple(),
    outcomes=('left', 'right'), 
    pmf_array = hand_pmf_array
)

jacket_pmf_array = np.array(
        [
            [0.9,0.8,0.1,0.3,0.4,0.01,0.02,0.2],
            [0.08,0.17,0.03,0.35,0.05,0.01,0.2,0.08],
            [0.02,0.03,0.87,0.35,0.55,0.98,0.78,0.72],
        ]
    )
jacket = RandomVariable(
    name='jacket',
    parents=(region, gender),
    outcomes=('full', 'part', 'never'), 
    pmf_array = jacket_pmf_array
)

age_pmf_array = np.array(
        [
            [0.01,0.96,0.3],
            [0.98,0.03,0.5],
            [0.01,0.01,0.2],
        ]
    )
age = RandomVariable(
    name='age',
    parents=(jacket, ),
    outcomes=('new', 'worn', 'old'), 
    pmf_array = age_pmf_array
)

colour_pmf_array = np.array(
        [
            [0.9,0.83,0.17,0.3],
            [0.1,0.17,0.83,0.7],
        ]
    )
colour = RandomVariable(
    name='colour',
    parents=(gender, hand),
    outcomes=('black', 'white'),
    pmf_array = colour_pmf_array
)

vote = RandomVariable(
    name='vote',
    parents=(region, age, colour),
    outcomes=vote_outcomes,
    pmf_array = vote_pmf_array
)


election_model = BayesianNetwork(region, gender, hand, jacket, age, colour, vote)

### 1A (1 mark) Joint distribution of a subset

- Compute the joint distribution of **Jacket**, **Region** and **Gender**. Print in a tab separated format the resulting distribution.

> We are computing the joint distribution simply by using the product rule

In [5]:
## We are computing the joint distribution simply by using the product rule
p_j_r_g = np.multiply (np.hstack (region_pmf_array.dot (gender_pmf_array.T)), jacket_pmf_array)
# print (p_j_r_g)
print ('P (J , r , g)')
print ('\t','%-11s %-11s %-11s %-11s %-11s %-11s %-11s %-11s'%(tuple (itertools.product(region.outcomes, gender.outcomes))))
print ('J = full\t','%-11s %-11s %-11s %-11s %-11s %-11s %-11s %-11s'%(tuple (p_j_r_g[0])))
print ('J = part\t', '%-11s %-11s %-11s %-11s %-11s %-11s %-11s %-11s'%(tuple (p_j_r_g[1])))
print ('J = never\t', '%-11s %-11s %-11s %-11s %-11s %-11s %-11s %-11s'%(tuple (p_j_r_g[2])))

P (J , r , g)
	 ('north', 'male') ('north', 'female') ('south', 'male') ('south', 'female') ('east', 'male') ('east', 'female') ('west', 'male') ('west', 'female')
J = full	 0.054       0.112       0.003       0.021       0.06        0.0035      0.0012      0.028      
J = part	 0.0048      0.0238      0.0009      0.0245      0.0075      0.0035      0.012       0.0112     
J = never	 0.0012      0.0042      0.0261      0.0245      0.0825      0.343       0.0468      0.1008     


### 1B1 (2 marks) Variable Ordering

1. Implement a function which determines an appropriate ordering of the variables for use in the following scheme:
    - For the first node R, draw a sample from p(R).
    - For each subsequent node, draw a sample from the conditional distribution $p(X \,|\, \text{parents}(X))$ where $\text{parents}(X)$ are the parents of the variable $X$ in the graphical model.
- Use your function to compute such an ordering and print the result in a human friendly format.

In [6]:
def order (network):
    ## This function calculates a partial order for bayesian networks
    ## returns list of random variables
    ## Returns None if there's a cycle in the network
    ordered_vars = list ()
    traversed = set ()
    while not len (traversed) == len (network.random_variables):
        old_length = len (traversed)
        for node in network.random_variables:
            if (node.parents == tuple() or set (node.parents).issubset(traversed)) and node not in traversed:
                traversed.add (node)
                ordered_vars.append (node)
        # If graph is cyclic return "None"
        if old_length == len (traversed) and not len (traversed) == len (network.random_variables):
            return None
    return ordered_vars

# Now let's calculate the order for our 'election_model' network.
o_election = order (election_model)

In [7]:
# Let's print our random variables in the determined order
st = "\n".join(map (str, o_election))
print (st)

region -> ('north', 'south', 'east', 'west')
gender -> ('male', 'female')
hand -> ('left', 'right')
jacket -> ('full', 'part', 'never')
age -> ('new', 'worn', 'old')
colour -> ('black', 'white')
vote -> ('bernie', 'donald', 'hillary', 'ted')


### 1B2 (2 marks) Sampling

1. Given the ordering you have determined above, implement the sampler described in the previous question. If you were unable to compute the ordering in the previous question, use ``ordering = (hand, region, gender, colour, jacket, age, vote)``.
2. Draw a single sample and print the result in a human friendly format.

In [8]:
def sampler (ordered_network):
    ## This function accepts an ordered network (i.e. list of random variables), and returns a sample from our network
    ## 'samples' stores the sample for each node in our network
    samples = dict ()
    for node in ordered_network.random_variables:
        parent_names = [c.name for c in node.parents]
        parent_values = tuple ([samples [index] for index in parent_names])
        current_choice = np.random.choice (a = list (node.pmfs [parent_values].keys()), 
                                           p = list (node.pmfs [parent_values].values ()), size = 1)[0]
        samples [node.name] = current_choice
    return samples

In [9]:
## Let's draw a sample from our network
ordered_election = BayesianNetwork (*o_election)
sample = sampler (ordered_election)
print (sample)

{'age': 'new', 'vote': 'donald', 'colour': 'white', 'hand': 'right', 'region': 'north', 'jacket': 'part', 'gender': 'male'}


### 1B3 (2 marks) Marginals

1. Calculate (and show in LaTeX) the marginal distribution for **Jacket**.
- Implement a function which computes the marginal distribution of each variable. It should make use of the ordering used by your sampler.
- Implement a function which takes a list of samples the model and computes the empirical marginal distribution of each variable.
- Plot the theoretical and approximate marginals which you obtain along with the absolute percent error between the two, in the format below:

## 1.
> By sum rule, we can write $p (J = x)$ as:

$$ p (J=x) = \sum_{G}\sum_{R} p (J = x, R = r, G = g) $$

> And by product rule we can extend the joint probability to the product of conditional and marginal probabilities given to us

$$ p (J = x, R = r, G = g) = p (J = x | R = r, G = g) p(R = r|G = g) p (G = g)$$

> So for each $x \in outcomes (J)$ we have

$$ p (J = x) = \sum_{G}\sum_{R}p (J = x | R = r, G = g) p(R = r|G = g) p (G = g)$$

> To do so, we use the previously calculated joing probability and marginalise $G$ and $R$ by summing over the rows.

In [10]:
## To do so
p_jacket = np.sum (p_j_r_g, axis=1)
print ('P of jacket\n')
print ('%-20s %-20s %-20s'%('full', 'part', 'never'))
print ('%-20s %-20s %-20s'% tuple(p_jacket.tolist ()))

P of jacket

full                 part                 never               
0.28269999999999995  0.08819999999999999  0.6291              


### 2
Marginal distribution of each variable

In [11]:
# def calculate_probabilities (region, jacket, gender, color, hand, age, vote):
def calculate_probabilities (network, joint_prob_dict):
    ## This function calculates the probability for a sample from all our random variables
    ## Given the network and a dictionary of the samples for each probability
    probability = 1
    for rv in network.random_variables:
        probability *= rv.pmfs [tuple ([joint_prob_dict[i.name] for i in rv.parents])][joint_prob_dict [rv.name]]
    return probability

def calculate_marginal_outcome (outcome, network):
    ## Get all the outcomes, all the names and all the variables
    variables_outcome = {var.name:var.outcomes for var in network.random_variables}
    variable_names = [var.name for var in network.random_variables]

    ## List of lists
    ## It contains a list for each random variable which contains a list of the possible outcomes
    ## Basically same as 'variable_outcomes', except that it's a list and therefore is ordered/
    list_of_variable_outcomes = [variables_outcome [key] for key in variable_names]
    ## All possible combinations of outcomes
    variables = itertools.product (*list_of_variable_outcomes)
    ## We've passed an outcome, and we're refining our variables to get everything in it that includes 'outcome' in a list
    variable_list = [var for var in variables if outcome in var]
    # We start from 0 and sum over whatever we're marginalising
    marginal= 0
    for var in variable_list:
        ## Zip all the variables and values into a dict and add it to the marginal probability
        variable_dict = dict (zip (variable_names, var))
        marginal += calculate_probabilities (network, variable_dict)
    return marginal

def calculate_marginal_variable (variable, ordered_network):
    ## Given a random variable, finds the probabilities for all the possible outcomes of that probability
    return {outcome : calculate_marginal_outcome (outcome, ordered_network) for outcome in variable.outcomes}

def calculate_marginal_network (ordered_network):
    ## Create a dictionaty with all the random variables in the network as keys and 
    ## a dict storing marginal probabilities for each outcome
    return {rv.name : calculate_marginal_variable (rv, ordered_network) for rv in ordered_network.random_variables}



In [12]:
# Call the function, let's print it in a human readable but ugly format
print ('Marginal probabilities => theoretical')
marginal = calculate_marginal_network (ordered_election)
for rv in marginal.keys ():
    print (rv, ': ')
    for marg in marginal [rv].keys ():
        print (marg,': ', marginal [rv][marg], ' \t', end='')
#     print ((marginal [rv]))
    print ('\n')
        

Marginal probabilities => theoretical
age : 
new :  0.276229  	worn :  0.594242  	old :  0.129529  	

vote : 
bernie :  0.15481761532  	ted :  0.20465624916  	hillary :  0.29550527822  	donald :  0.3450208573  	

colour : 
black :  0.396  	white :  0.604  	

hand : 
right :  0.1  	left :  0.9  	

region : 
south :  0.1  	east :  0.5  	west :  0.2  	north :  0.2  	

jacket : 
part :  0.0882  	full :  0.2827  	never :  0.6291  	

gender : 
female :  0.7  	male :  0.3  	



In [13]:
def calculate_empirical_distribution (network, samples):
    ## Get the names of random variables (from our ordered network)
    rv_names = [rv.name for rv in network.random_variables]
    ## And the outcomes
    rv_outcomes = {rv.name : rv.outcomes for rv in network.random_variables}
    marginal = dict ()
    n = len (samples)
    ## Go through all random variables
    for rv in rv_names:
        ## Create empty dicts for each rv if doesn't already exist
        if rv not in marginal:
            marginal [rv] = dict ()
        for outcome in rv_outcomes [rv]:
            ## Add key with value 0 to the dictionary for every possible outcome of that RV
            marginal [rv][outcome] = 0
    for sample in samples:
        for rv in rv_names:
            ## Increment the occurence of that outcome by one
            marginal [rv][sample [rv]] += 1
    ## Go through all random variables and divide each of them by the number of samples
    for rv in rv_names:
        for key in marginal [rv].keys ():
            ## Divide everything by total number of samples
            marginal [rv][key] /= n
    return marginal

In [14]:
my_samples = list ()
for i in range (100000):
    my_samples.append (sampler (election_model))

marginal_empirical = calculate_empirical_distribution (election_model, my_samples)

In [15]:
print ('Marginal probabilities => empirical\n')
for item in marginal_empirical.keys ():
    print (item)
    for outcome in marginal_empirical [item].keys ():
        print (outcome, ': ', marginal_empirical [item][outcome], '\t', end='')
    print ('\n')

Marginal probabilities => empirical

age
new :  0.27535 	worn :  0.59384 	old :  0.13081 	

vote
bernie :  0.15512 	ted :  0.20486 	hillary :  0.296 	donald :  0.34402 	

colour
black :  0.39588 	white :  0.60412 	

hand
right :  0.09991 	left :  0.90009 	

region
south :  0.09932 	east :  0.50285 	west :  0.19919 	north :  0.19864 	

jacket
part :  0.08774 	full :  0.28199 	never :  0.63027 	

gender
female :  0.70059 	male :  0.29941 	



In [16]:
for rv in election_model.random_variables:
    emp = marginal_empirical [rv.name]
#     print (marginal [rv.name])
    marg = marginal [rv.name]
    print (rv.name)
    print ('%-20s %-20s %-20s %-20s' %('outcome','exact', 'approx', 'error (%)'))
    for outcome in emp.keys ():
        print ('%-20s %-20s %-20s %-20s' %(outcome, marg [outcome], emp [outcome], np.abs (emp [outcome] - marg [outcome])*100/marg [outcome]))
    print()
               

region
outcome              exact                approx               error (%)           
south                0.1                  0.09932              0.68                
east                 0.5                  0.50285              0.57                
west                 0.2                  0.19919              0.405               
north                0.2                  0.19864              0.68                

gender
outcome              exact                approx               error (%)           
female               0.7                  0.70059              0.0842857142856     
male                 0.3                  0.29941              0.196666666667      

hand
outcome              exact                approx               error (%)           
right                0.1                  0.09991              0.0900000000001     
left                 0.9                  0.90009              0.00999999999995    

jacket
outcome              exact                appro

> The order for P (Hand) is the reverse of what is in the tables. Here, I'm assuming the values used to initialise the random variables are correct.

### 1B4 (1 mark) Easy conditional probabilities

Compute $p(X \,|\, G=\text{female})$ for all $X$ other than $G$,
1. Approximately, using your sampler
- Exactly, using your marginal calculating function from the previous question. Hint: what happens if you set  $p(G=\text{female})=1$ in your model?
- Plot the results side by side in the same format as the previous question.
- State for which variables other than $G$ the theoretical scheme above can be used to compute such conditionals, and why.

In [17]:
def p_x_given_female (samples):
    ## We get bunch of samples, if they're female we look at them and
    ## Count the number of occurance of each outcome
    marginal = dict ()
    n_female = 0
    for sample in samples:
        if sample ['gender'] == 'female':
            n_female += 1
            for rv in sample:
                if rv not in marginal:
                    marginal [rv] = dict ()
                if rv != 'gender':
                    if sample [rv] not in marginal [rv]:
                        marginal [rv][sample [rv]] = 0
                    marginal [rv][sample [rv]] += 1
                else:
                    marginal [rv]['male'] = 0
                    marginal [rv]['female'] = n_female
    ## And in the end we divide everything by total number of samples where Gender = female
    for rv in marginal.keys ():
        for outcome in marginal [rv].keys ():
            marginal [rv][outcome] /= n_female
    return marginal

my_samples = list ()
for i in range (200):
    my_samples.append (sampler (election_model))

x_female_exp = p_x_given_female (my_samples)


In [18]:
## Same thing but not with sampling. Exact calculation
network_female_1 = copy.deepcopy (election_model)
for rv in network_female_1.random_variables:
    if rv.name == 'gender':
        rv.pmfs [()] = {'male':0, 'female':1}
        break
marginal_f_1 = calculate_marginal_network (network_female_1)


In [19]:
for rv in network_female_1.random_variables:
    print (rv.name)
    print ('%-20s %-20s %-20s %-20s' %('outcome','exact', 'approx', 'error (%)'))
    for outcome in rv.outcomes:
        print ('%-20s %-20s %-20s %-20s' %(outcome, marginal_f_1 [rv.name][outcome], x_female_exp [rv.name][outcome],\
                                          np.abs (marginal_f_1 [rv.name][outcome]- x_female_exp [rv.name][outcome])*100/\
                                          1 if marginal_f_1[rv.name][outcome] == 0 else marginal_f_1[rv.name][outcome]))
    print ()

region
outcome              exact                approx               error (%)           
north                0.2                  0.2391304347826087   0.2                 
south                0.1                  0.09420289855072464  0.1                 
east                 0.5                  0.5217391304347826   0.5                 
west                 0.2                  0.14492753623188406  0.2                 

gender
outcome              exact                approx               error (%)           
male                 0.0                  0.0                  0.0                 
female               1.0                  1.0                  1.0                 

hand
outcome              exact                approx               error (%)           
left                 0.9                  0.8840579710144928   0.9                 
right                0.1                  0.11594202898550725  0.1                 

jacket
outcome              exact                appro

>  The theoritical scheme above can be used for all variables that have no parents in our graphical model. i.e. variables that are conditionaly independent from other variables. Hence, in this particular example other than $G$, we can apply the above scheme to $Region$, and $Hand$. This is because we can simply change their probabilities to imply the given condition while for all other nodes our probabilities depend on other variables.

### 1B5 (3 marks) General conditional probabilities

1. Write down the expression of the joint probability $p(R,G,H,J,A,C,V)$ in terms of the conditional probabilities in the graphical model.
- Derive $p(G = male \,|\, V = Donald)$ in terms of the conditional probabilities of the graphical model.
- Compute and display in a human friendly format the conditional distributions $p(G=g \,|\, V=v)$ for all genders $g$ and votes $v$, by naively marginalising the other variables (other than $G$ and $V$, that is).

### Solution description

## 1.
> From our graphical model, we know that:

$$ p (V, A, J, R, G, C, H) = p (V | R, A, C)\times p (R) \times p (A | J) \times p (J | R, G) \times p (C | G, H) \times p (G) \times p (H)$$

## 2.
> In order to calculate $p (G = male | V = Donald)$ we first write it down as the following:

$$ \frac {p (G = male, V = Donald)}{p (V = Donald)}$$

> We can then calculate the numerator of the above formula by marginalising out the variables that we do not need.

$$ \sum_{R, H, J, C, A} p (V = Donald, A, J, R, G = male, C, H) = \sum_{R, H, J, C, A} p (V = Donald | R, A, C)\times p (R) \times p (A | J) \times p (J | R, G = male) \times p (C | G = male, H) \times p (G = male) \times p (H)$$

> And to save some calculation, we write the above as

$$ p (G = male)\sum_{R, H, J, C, A} p (V = Donald | R, A, C)\times p (R) \times p (A | J) \times p (J | R, G = male) \times p (C | G = male, H) \times p (H)$$

> So we write $p (G = male| V = Donald)$ as

$$ \frac{p (G = male)\sum_{R, H, J, C, A} p (V = Donald | R, A, C)\times p (R) \times p (A | J) \times p (J | R, G = male) \times p (C | G = male, H) \times p (H)}{p (V = Donald)}$$

> To save computation we can push the sums in

$$ \frac{p (G = male)\sum_H(\sum_{C, A}(\sum_R(\sum_J p (J | R, G = male)\times p (A | J))\times p (C | G = male, H))\times p (R) \times p (V = Donald | R, A, C))\times p (H))}{}$$

## 3

In [20]:
## Let's calculate p(G=g|V=v) for all genders and votes
def calculate_p_g_given_v (network):
    index = 0;
    vote_index, gender_index, gender_outcomes = None, None, None;
    for rv in network.random_variables:
        if rv.name == 'vote':
            vote_index = index
        elif rv.name == 'gender':
            gender_index = index
            gender_outcomes = rv.outcomes
        index += 1
    ## Let's create a dictionary to store our probabilities and get the votes to use with bayes rule
    probabilities = dict ()
    p_vote = calculate_marginal_network (network)['vote']
    
    ## Create a new dictionary to store the new conditional variables
    ordered_probabilities = dict ()
    ## Go through all gender outcomes
    for outcome in gender_outcomes:
        p_gender = network.random_variables [gender_index].pmfs [()][outcome]
        new_network = copy.deepcopy (network)
        ## Now go through all vote outcomes
        for pmf in new_network.random_variables [gender_index].pmfs.keys ():
            ## Change the probabilities as we want to calculate conditional probabilities
            for current_outcome in gender_outcomes:
                ## If we're conditioning on a certain outcome, we want its probability to be 1
                ## and otherwise we want it to be 0
                new_network.random_variables [gender_index].pmfs [pmf][current_outcome] = 1 if outcome == current_outcome else 0
                ## New network created given V = outcome
        ## Using our new network, we want to marginalise a variable,
        ## so we call the function that we had implemented for 1B3
        probabilities [outcome] = calculate_marginal_variable (ordered_network = new_network, 
                                                               variable = new_network.random_variables [vote_index])
        ## So far we have p (V | G)
        ## Let's go through votes and apply bayes rule to get p (G | V)
    
        for key in p_vote.keys ():
            probabilities [outcome][key] = probabilities [outcome][key]*p_gender/p_vote[key]
            ## And assign in to the new duictionary with the right order
            if key not in ordered_probabilities:
                ordered_probabilities [key] = dict ()
            ordered_probabilities [key][outcome] = probabilities [outcome][key]
        
    return ordered_probabilities

g_given_v = calculate_p_g_given_v (ordered_election)

In [21]:
# Given vote, probability of gender
for item in g_given_v.keys ():
    print ('given ', item)
    print ('probability of ', g_given_v [item])
    print ()

given  bernie
probability of  {'female': 0.60186700981928043, 'male': 0.3981329901807199}

given  ted
probability of  {'female': 0.72156242458323416, 'male': 0.27843757541676628}

given  hillary
probability of  {'female': 0.63644447007130023, 'male': 0.36355552992870016}

given  donald
probability of  {'female': 0.78567837136378282, 'male': 0.21432162863621748}



### 1B6 (2 marks) Variable elimination

Denote the graphical model consider thus far $\mathcal{M}$.

1. Derive $p(R,G,J,A,C,V)$ by marginalising over $H$ in $\mathcal{M}$. 
- Describe how the structure (connectivity) of the new graphical model (call it $\mathcal{M}_H$) over all variables other than $H$ changes after eliminating $H$ in this way.
- Describe which conditional(s) in the new graphical model differ from the original model.
- Encode the $\mathcal{M}_H$ in python similarly to $\mathcal{M}$. 

### 1
$$ p (R, G, J, A, C, V) = \sum_{H}p (R, G, H, J, A, C, V)$$

> Therefore according to the joint probability in the last question we can write the above expression as

$$ \sum_{H} p (V | R, A, C)\times p (R) \times p (A | J) \times p (J | R, G) \times p (C | G, H) \times p (G) \times p (H)$$

$$ p (V | R, A, C)\times p (R) \times p (A | J) \times p (J | R, G) \times p (G) \sum_{H} p (C | G, H) \times p (H)$$

### 2

> As the only dependent variable on $H$ was $C$, we merely need to omit $H$ and marginalise it in $p (C|H, G)$. This will result in the following $p (C | G)$

$$ p (C | G) = \sum_H\frac{p (C | G, H) \times p (H) * p (G)}{p (G)} = \sum_H p (C | G, H) \times p (H)$$

> Therefore we will eliminate $H$ and calculate $p (C | G)$ by marginalising $H$ using the above formula

> And finally we will get


$$ p (V | R, A, C)\times p (R) \times p (A | J) \times p (J | R, G) \times p (G) \times p (C | G) $$

### 3

> As it was stated in the last answer, $p (H)$ will not exist in the new network and $p (C | G, H)$ will turn into $p (C | G)$

In [22]:
## Marginalise H
## This will be our new network
ordered_marginalised_H_election = copy.deepcopy (ordered_election)

## Get the indices of the random variables we will be using
hand_index, colour_index, gender_index, index = None, None, None, 0
for rv in ordered_marginalised_H_election.random_variables:
    if rv.name == 'hand':
        hand_index = index
    if rv.name == 'colour':
        colour_index = index
    if rv.name == 'gender':
        gender_index = index
    index += 1

## marginalise H in p (C | G, H) to derive the new p (C | G)
p_color_given_gender = dict ()
for gender_outcome in ordered_marginalised_H_election.random_variables [gender_index].outcomes:
    p_color_given_gender [(gender_outcome, )] = dict ()
    for hand_outcome in ordered_marginalised_H_election.random_variables [hand_index].outcomes:
        for colour_outcome in ordered_marginalised_H_election.random_variables [colour_index].outcomes:
            if colour_outcome not in p_color_given_gender [(gender_outcome, )]:
                p_color_given_gender [(gender_outcome, )][colour_outcome] = 0
            p_color_given_gender [(gender_outcome, )][colour_outcome] += \
                ordered_marginalised_H_election.random_variables [colour_index].pmfs [(gender_outcome, hand_outcome)]\
                [colour_outcome]* ordered_marginalised_H_election.random_variables [hand_index].pmfs [()][hand_outcome]
                

## Print out what we just calculated
for gender in p_color_given_gender.keys ():
    print ('given ', gender)
    for color in p_color_given_gender [gender].keys ():
        print (color, ': ', p_color_given_gender [gender][color], '\t', end = '')
    print ()

## Now let's change the old network to create the new one
# first off, we change the pmfs for the random variable color
ordered_marginalised_H_election.random_variables [colour_index].pmfs = p_color_given_gender
# python bs. stupid language, annoying type system.
ordered_marginalised_H_election.random_variables = list (ordered_marginalised_H_election.random_variables)
parents = list (ordered_marginalised_H_election.random_variables [colour_index].parents)
# find index of hand in the random variables
hand_index = 0
for parent in parents:
    if parent.name == 'hand':
        break
    hand_index += 1
    
# delete it from the parents
del parents [hand_index]

# We'll need the random variable gender, let's find it then
g = None
for var in ordered_marginalised_H_election.random_variables [colour_index].parents:
    if var.name == 'gender':
        g = var
        break
# In the new network colour has a single parent. i.e. gender
ordered_marginalised_H_election.random_variables [colour_index].parents = (g, )

# Get rid of hand in the list of random variables
# Why +1? python bs.
ordered_marginalised_H_election.random_variables.pop (hand_index+1)
ordered_marginalised_H_election.random_variables = tuple (ordered_marginalised_H_election.random_variables)
# print (ordered_marginalised_H_election.random_variables)

given  ('female',)
black :  0.183 	white :  0.817 	
given  ('male',)
black :  0.893 	white :  0.107 	


In [23]:
# Call the function, let's print it in a human readable but ugly format
print ('Marginal probabilities for our new network => theoretical')
marginal = calculate_marginal_network (ordered_marginalised_H_election)
for rv in marginal.keys ():
    print (rv, ': ')
    for marg in marginal [rv].keys ():
        print (marg,': ', marginal [rv][marg], ' \t', end='')
    print ('\n')

Marginal probabilities for our new network => theoretical
age : 
new :  0.276229  	worn :  0.594242  	old :  0.129529  	

vote : 
bernie :  0.15481761532  	ted :  0.20465624916  	hillary :  0.29550527822  	donald :  0.3450208573  	

colour : 
black :  0.396  	white :  0.604  	

region : 
south :  0.1  	east :  0.5  	west :  0.2  	north :  0.2  	

jacket : 
part :  0.0882  	full :  0.2827  	never :  0.6291  	

gender : 
female :  0.7  	male :  0.3  	



### 1B7 (3 marks) General estimation of conditional probabilities (revisited)

1. As you did earlier, compute and display in a human friendly format the conditional distributions $p(G=g \,|\, V=v)$ for all genders $g$ and votes $v$, by naively marginalising the other variables (other than $G$ and $V$, that is). This time however, do so using $\mathcal{M}_H$ rather than $\mathcal{M}$.
- Quantify the computational advantages of using $\mathcal{M}_H$ in this way rather than $\mathcal{M}$.
- Which variable (or variables) would be the best to eliminate in this way, in terms of the aforementioned computational advantages, and why?
- Pick a (best) variable from the previous question and call it $X$. Assuming we have eliminated $X$, which would be the "best" variable to subsequently eliminate in a similar fashion?

In [24]:
import time

In [25]:
t0 = time.time ()
ordered_marg_H = calculate_p_g_given_v (ordered_marginalised_H_election)
print ('New network - time : ', time.time () - t0)

t0 = time.time ()
calculate_p_g_given_v (ordered_election)
print ('Normal network - time : ',time.time () - t0)
print ('-----------------')
for item in ordered_marg_H.keys ():
    print ('given ', item)
    print ('probability of ', g_given_v [item])
    print ()

New network - time :  0.07245254516601562
Normal network - time :  0.1189732551574707
-----------------
given  bernie
probability of  {'female': 0.60186700981928043, 'male': 0.3981329901807199}

given  ted
probability of  {'female': 0.72156242458323416, 'male': 0.27843757541676628}

given  hillary
probability of  {'female': 0.63644447007130023, 'male': 0.36355552992870016}

given  donald
probability of  {'female': 0.78567837136378282, 'male': 0.21432162863621748}



> As we can see and we expected, the results are the same and the time is reduced by 40%.

### 2
> To calculate $p (G | V)$ the naive way, we carry out the following operations.

[\\]: <> (> We change the probabilities for genders based on what is given to us.  *trivial*)


[\\]: <> (> \# of possible genders * (go through all combinations of all variables [perform addition] ** (\# of random variables)))

> Go through all possible genders

        >> Go through all combinations of random variables
        
        >>> Perform an addition
        
            >>>> Perform r multiplications where r is the number of random variables

> Therefore using $\mathcal{M}_H$ will result in:

> - 1 fewer random variable
> - new # of combinations = \# of combinations/# of outcomes of $H$

> Therefore, the best variable to eliminate first to reduce the time complexity and not just shift the complexity into the marginalisation will be the variable with the least number of random variables associated with it. 

> By association, we mean *parents of a node*, *its children*, and *its children's parents*.

> For instance, in the above example

$$ \sum_{H, R, J, A, C} p (V | R, A, C)\times p (R) \times p (A | J) \times p (J | R, G) \times p (C | G, H) \times p (G) \times p (H) $$

> turns in to

$$ \sum_{R, J, A, C} p (V | R, A, C)\times p (R) \times p (A | J) \times p (J | R, G) \times  p (G) \times \sum_{H}p (C | G, H) \times p (H) $$

> As we can see, $H$ has $2$ associated random variables; namely $G$ and $C$.

> By using the distributive law and pushing in the sum we are only carrying it out $n$ times where $n$ is the possible combinations of $G$, and $C$ rather than the above example in which this sum will be carried out as many times as the number of all possible combinations of $R$, $J$, $A$, $C$, $G$.

### 3
> As we stated before, we can express our model using the following probabilities:

$$ \sum_{R, J, A, C, H} p (V | R, A, C)\times p (R) \times p (A | J) \times p (J | R, G) \times  p (G) \times p (C | G, H) \times p (H) $$

> Looking for the variable with the least number of associated random variables and excluding $V$ and $G$, $H$ will be the best choice as it has the lowest number of random variables associated with it]; As we can see $H$ is only associated with $C$, and $G$. Other possible variables have the following associated variables.

$$R: V, A, C, J, G$$
$$A: R, C, V, J$$
$$C: R, V, A, G$$
$$J: A, R, G $$

> In other words the number of combinations will only depend on the outcomes of $H$, $C$, and $G$; therefore we can push the sum in only over what is relevant. Isolating the above variables and performing

$$ \sum_{R, J, A, C} p (V | R, A, C)\times p (R) \times p (A | J) \times p (J | R, G) \times  p (G) \times \sum_{H}p (C | G, H) \times p (H) $$

> will result in a reduction in the number of operations performed to calculate the probabilities and is similar to applying the distributive law to a simple sum of products where the first factor is the same.

[\\]: <> (> As a result, we will get the following model for our joint distribution)

[\\]: <> ($$ \sum_{R, A, C} p (V | R, A, C)\times p (R) \times  p (G) \times p (C | G) \times \sum_{J} p (A | J) \times p (J | R, G)$$)

### 4
> After eliminating $H$, we will have a new $p (C | G)$ and our model can be expressed as:

$$ \sum_{R, J, A, C} p (V | R, A, C)\times p (R) \times p (A | J) \times p (J | R, G) \times  p (G) \times p (C | G)$$

> As it was mentioned before, we are aiming at marginalising the variable that lets up push the sums in as far as possible. Such variable can be found by looking at the number of associated random variables with them and choosing the variable with the fewest associated variables.



> We saw that the next node with the least number of associated random variables is $J$. Therefore, by eliminating it and paying for the cost of marginalisation, we avoid redundant calculations (calculating the same thing more than once) and reduce our number of operations the most.

## Part 2: Theory

### 2A (3 marks) Functions of random variables

$u$ and $v$ are independently sampled from the standard uniform distribution on the unit interval, $[0,1]$. 

If $s=(2u-1)^2+(2v-1)^2 \geq 1$ then $x$ is sampled from the standard normal distribution, $\mathcal{N}(0,1)$. Otherwise $x=(2u-1)\sqrt{-2 \log(s)/s}$.

How is $x$ distributed?

### Solution description

> We want to prove that the two distributions are sampled from the same distribution to show that $p (x \sim \mathcal{N}(0, 1))$

> It is given to us that $p (x |  >= 1) \sim \mathcal{N}$ and we need to show

$$p (x | s < 1) \sim \mathcal{N}$$

> We know that the joint distribution of $x$ and $y$ is given by:

$$p (x, y) = P (U, V) |\frac{d(U, V)}{d (x, y)}|$$

> And by plugging in the chain rule we can get

$$ p (x, y) = p (U, V)|\frac{dUdV}{drd\theta}\frac{drd\theta}{dxdy}|$$

> And according to *http://tutorial.math.lamar.edu/Classes/CalcIII/ChangeOfVariables.aspx* using the standard jacobian for conversion to polar coordinates

> The above expresiion is equal to

$$ \frac{1}{\pi}\times r\times\frac{r}{2}$$

> To prove that out random variable is distributed normally, we introduce the new random variable $Y$, as well as $U$, and $V$ where

$$U (u) = 2i - 1$$

$$V (v) = 2v - 1$$

> And in polar coordinates, we have

$$ U (r, \theta) = r\cos{\theta}$$
$$ V (t, \theta) = r\sin{\theta}$$

> Hence, we can rewrite out $x$ and $y$ as:

$$ x = U \frac{\sqrt{-2\log{(U^2 + V^2)}}}{U^2 + V^2} = U \frac{\sqrt{-2 \log{r^2}}}{r^2}$$

$$ y = V \frac{\sqrt{-2 \log{r^2}}}{r^2}$$

> Since $S$ describes a circle, we know that its density is $\frac{1}{\pi}$. i.e. $p (U, V) = \frac{1}{\pi}$

> Hence

$$ P (x, y | s < 1) = \frac{r^2}{2\pi}$$

$$p (x, y| s < 1) = \int_{-\inf}^{\inf} \frac{1}{2\pi}e ^{\frac{-x^2+y^2}{2}}dy$$

$$ p (x, y | s < 1) = \frac{1}{\sqrt{2 \pi}}e^{\frac{-x^2}{2}}\int_{-\inf}^{\inf}e^\frac{-y^2}{2}dy$$

> And finally by marginalising $y$, we get

$$ p (x | s < 1) = \frac{1}{\sqrt{2\pi}}e^{\frac{-x^2}{2}}$$

> Which is a gausian and therefore

$$ p (x) \sim \mathcal{N}(0, 1)$$